<a href="https://colab.research.google.com/github/sosmany1/RNN_Parity-Problem/blob/master/RNN_Parity_relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math


num_epochs = 1000 
total_series_length = 400
state_size = 2
num_classes = 1 
batch_size = 100 
_learning_rate = 1e-1

        
def generateData():
    x = 1.0-2*(np.random.randn(batch_size,total_series_length) < 0).astype(np.float32) 
    y = np.zeros((batch_size,total_series_length),np.float32) 
    for i in range(total_series_length):
      y[:, i] = np.prod(x[:,0:(i+1)],axis=1)
    x[x==1] = 0
    x[x==-1] = 1
    y[y==1] = 0
    y[y==-1] = 1
    
    x = x.reshape((batch_size, -1))  
    y = y.reshape((batch_size, -1))
    return x, y

batchX_placeholder = tf.placeholder(tf.float32, [batch_size, total_series_length]) 
batchY_placeholder = tf.placeholder(tf.float32, [batch_size, total_series_length]) 

inputs_series = tf.unstack(value=batchX_placeholder, axis=1, num=total_series_length) 
labels_series = tf.unstack(value=batchY_placeholder, axis=1, num=total_series_length) 

inputs_series = [tf.expand_dims(current_input,axis=1) for current_input in inputs_series]
labels_series = [tf.expand_dims(current_label, axis=1) for current_label in labels_series]

#W = tf.Variable(tf.truncated_normal([2, state_size], stddev=1.0 / math.sqrt(float(2))), name='weights1')
#b = tf.Variable(tf.zeros([state_size]),name='biases1')

#W2 = tf.Variable(tf.truncated_normal([state_size, 1], stddev=1.0 / math.sqrt(float(state_size))), name='weights2')
#b2 = tf.Variable(tf.zeros([1]),name='biases2')


W = tf.Variable([[1,1],[1,1]], dtype=tf.float32, name='weights1')
b = tf.Variable([-1.5, -0.5], dtype=tf.float32, name='biases1')
W2 = tf.Variable([[-1],[1]], dtype=tf.float32, name='weights2')
b2 = tf.Variable([-0.5], dtype=tf.float32,name='biases2')


prev_logit = tf.zeros([batch_size, 1])
states_series = []
logits_series = []
for current_input in inputs_series:
  
    current_input = tf.reshape(current_input, [batch_size, 1])
    prev_logit = tf.reshape(prev_logit, [batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, prev_logit],1) 
    
    next_state = tf.nn.relu(tf.matmul(input_and_state_concatenated, W) + b)  
    states_series.append(next_state)    
    
    prev_logit = tf.matmul(next_state, W2) + b2 
    logits_series.append(prev_logit)


losses = [(tf.reduce_mean(tf.nn.relu(1.0 - logits*labels))) for logits, labels in zip(logits_series,labels_series) ]  
total_loss = tf.reduce_mean(losses)

#train_step = tf.train.GradientDescentOptimizer(learning_rate=_learning_rate).minimize(total_loss)

accuracy = tf.reduce_mean(tf.cast(labels_series[-1]*logits_series[-1]>0,tf.float32))

  
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    plt.ion()#
    plt.figure()#
    plt.show()#
    loss_list = []#
    accuracy_list = []
    for epoch_idx in range(num_epochs):
      x,y = generateData()
      #_total_loss, _,_acc = sess.run(
      _total_loss,_acc = sess.run(
          #[total_loss, train_step, accuracy],
          [total_loss, accuracy], # this gets rud if training
          feed_dict={
              batchX_placeholder:x,
              batchY_placeholder:y
          })
      loss_list.append(_total_loss)
      accuracy_list.append(_acc)
      plt.subplot(1, 2, 1)
      plt.plot(loss_list)
      plt.subplot(1, 2, 2)
      plt.plot(accuracy_list)

      #print("Step",epoch_idx, "Accuracy Last Time Step", _acc)
      print("Loss",_total_loss, "Accuracy Last Time Step", _acc)
plt.ioff()
plt.show()

[[-1.  1.  1. ...  1. -1.  1.]
 [-1. -1.  1. ... -1.  1.  1.]
 [ 1. -1.  1. ... -1.  1.  1.]
 ...
 [-1.  1. -1. ...  1. -1. -1.]
 [ 1.  1. -1. ... -1.  1.  1.]
 [ 1.  1. -1. ... -1. -1. -1.]]
Loss 1.000675 Accuracy Last Time Step 0.54
[[ 1. -1.  1. ... -1. -1. -1.]
 [ 1.  1. -1. ...  1. -1.  1.]
 [-1. -1.  1. ...  1. -1. -1.]
 ...
 [ 1.  1.  1. ...  1. -1.  1.]
 [ 1.  1. -1. ...  1. -1. -1.]
 [-1.  1. -1. ...  1. -1.  1.]]
Loss 0.997625 Accuracy Last Time Step 0.58
[[ 1.  1. -1. ... -1.  1.  1.]
 [ 1. -1. -1. ...  1. -1.  1.]
 [ 1. -1. -1. ... -1. -1.  1.]
 ...
 [ 1. -1. -1. ...  1. -1. -1.]
 [ 1.  1.  1. ...  1. -1. -1.]
 [ 1.  1. -1. ...  1. -1.  1.]]
Loss 0.9950499 Accuracy Last Time Step 0.51
[[ 1. -1. -1. ... -1.  1.  1.]
 [ 1.  1.  1. ...  1. -1. -1.]
 [-1. -1.  1. ... -1. -1.  1.]
 ...
 [-1.  1.  1. ...  1.  1. -1.]
 [ 1. -1.  1. ... -1. -1.  1.]
 [ 1.  1.  1. ...  1.  1. -1.]]
Loss 0.99459994 Accuracy Last Time Step 0.47
[[ 1.  1. -1. ... -1. -1.  1.]
 [-1.  1.  1. ... -1.  1. 

KeyboardInterrupt: ignored